<a href="https://colab.research.google.com/github/MaheshbabuAmuda/NNDL/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment-2**

-20K45A0229.

#Import required libraries

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

##Import Data

In [22]:
data = pd.read_excel('/content/Load Data in kW.xlsx')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


#Data normalization

In [23]:
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i - 24 * 7])
    y.append(data['Load (kW)'].iloc[i])
data = pd.DataFrame({'Previous Week Load' : x, 'Present Week Load' : y})
data.rename(columns={'Previous Week Load': 'X',
                   'Present Week Load': 'Y'},
          inplace=True)
data.head()

,X,Y
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


In [24]:
scaler = MinMaxScaler()
num_vars = ['X', 'Y']
data[num_vars] = scaler.fit_transform(data[num_vars])
data

,X,Y
0,0.397516,0.360304
1,0.293377,0.322302
2,0.276021,0.286040
3,0.309820,0.254087
4,0.332401,0.300562
...,...,...
2011,0.499943,0.228524
2012,0.536940,0.217797
2013,0.622923,0.246298
2014,0.489837,0.324384


##Data split to training and testing

In [25]:
x_train, x_test, y_train, y_test = train_test_split(data.X, data.Y, test_size = 0.1, random_state = 10)

##Initialization of required variables

In [26]:
m = 1
c = -1
eta = 0.01
max_iters = 100
r = 0.1

##Momentum + SGD Algorithm

In [27]:
m_deriv = 0
c_deriv = 0
Vm = 0
Vc = 0
for iter in range(max_iters):
  i = 0
  for i in range(len(data)):
    x = data.X[i]
    y = data.Y[i]
    m_deriv = -(y-m*x-c)*x
    c_deriv = -(y-m*x-c)
    Vm = r*Vm - eta*m_deriv
    Vc = r*Vc - eta*c_deriv
    m = m + Vm
    c = c + Vc
print("Model parameter m and c are : ",m,",",c)

Model parameter m and c are :  0.4453299332140738 , 0.14706183761212027


##Model prediction on training data

In [28]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m * i) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

##Model prediction on testing data

In [29]:
y_pred_test = []
for i in x_test:
  y_p_te = (m * i) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

##Error metrics calculation

In [30]:
#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error = ', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error = ', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error = ', mae)

Root mean square error =  0.20711212887043964
Mean square error =  0.0428954339252456
Mean absolute error =  0.1640759701454537


In [31]:
#Testing Accuracies
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error = ', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error = ', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error = ', mae)

Root mean square error =  0.20991236476513872
Mean square error =  0.04406320088129265
Mean absolute error =  0.16763584288533132


##Deployment

In [32]:
X=input('enter X value : ')
print('X = ',X)
Y=m*float(X)+c
print('Y = ',Y)

enter X value : 2000
X =  2000
Y =  890.8069282657598
